In [ ]:
import numpy as np
import cv2
import os
import math
import tensorflow as tf
from IPython.display import display, clear_output

def DrawBoundingBoxes(img, grid, threshold=0):
    image_h = img.shape[0]
    image_w = img.shape[1]
    grid_h = grid.shape[0]
    grid_w = grid.shape[1]
    cell_size_x = image_w // grid_w
    cell_size_y = image_h // grid_h
    confidence_map = grid[:, :, 4]
    cells = np.argwhere(confidence_map > threshold)
    clear_output(wait=True)
    print(np.max(confidence_map))
    bounding_boxes = []
    for cell in cells:
        bb_x = math.floor((cell[1] + grid[cell[0], cell[1], 1])*cell_size_x)
        bb_y = math.floor((cell[0] + grid[cell[0], cell[1], 0])*cell_size_y)
        bb_width = math.floor(grid[cell[0], cell[1], 3] * image_w)
        bb_height = math.floor(grid[cell[0], cell[1], 2] * image_h)
        bounding_boxes.append((bb_x, bb_y, bb_width, bb_height))
        cv2.rectangle(img, (bb_x-bb_width//2, bb_y-bb_height//2), (bb_x+bb_width//2, bb_y+bb_height//2), (0,255,0), 1)
    return img

def model(x, filt=32, use_batch_norm=True, alpha=0.1):
    Conv1 = tf.layers.conv2d(inputs=x, filters=filt, kernel_size=[5, 5], strides = 1, padding="same", activation=None, name = "Conv1")
    if use_batch_norm:
        Conv1 = tf.contrib.layers.batch_norm(inputs = Conv1)
    Conv1 = tf.maximum(Conv1, alpha*Conv1)
    Conv2 = tf.layers.conv2d(inputs=Conv1, filters=filt, kernel_size=[5, 5], strides = 2, padding="same", activation=None, name = "Conv2")
    if use_batch_norm:
        Conv2 = tf.contrib.layers.batch_norm(inputs = Conv2)
    Conv2 = tf.maximum(Conv2, alpha*Conv2)
    
    Conv3 = tf.layers.conv2d(inputs=Conv2, filters=2*filt, kernel_size=[5, 5], strides = 1, padding="same", activation=None, name = "Conv3")
    if use_batch_norm:
        Conv3 = tf.contrib.layers.batch_norm(inputs = Conv3)
    Conv3 = tf.maximum(Conv3, alpha*Conv3)
    Conv4 = tf.layers.conv2d(inputs=Conv3, filters=2*filt, kernel_size=[5, 5], strides = 2, padding="same", activation=None, name = "Conv4")
    if use_batch_norm:
        Conv4 = tf.contrib.layers.batch_norm(inputs = Conv4)
    Conv4 = tf.maximum(Conv4, alpha*Conv4)
    
    Conv5 = tf.layers.conv2d(inputs=Conv4, filters=4*filt, kernel_size=[3, 3], strides = 1, padding="same", activation=None, name = "Conv5")
    if use_batch_norm:
        Conv5 = tf.contrib.layers.batch_norm(inputs = Conv5)
    Conv5 = tf.maximum(Conv5, alpha*Conv5)
    Conv6 = tf.layers.conv2d(inputs=Conv5, filters=4*filt, kernel_size=[3, 3], strides = 2, padding="same", activation=None, name = "Conv6")
    if use_batch_norm:
        Conv6 = tf.contrib.layers.batch_norm(inputs = Conv6)
    Conv6 = tf.maximum(Conv6, alpha*Conv6)
    
    Conv7 = tf.layers.conv2d(inputs=Conv6, filters=8*filt, kernel_size=[3, 3], strides = 1, padding="same", activation=None, name = "Conv7")
    if use_batch_norm:
        Conv7 = tf.contrib.layers.batch_norm(inputs = Conv7)
    Conv7 = tf.maximum(Conv7, alpha*Conv7)
    Conv8 = tf.layers.conv2d(inputs=Conv7, filters=8*filt, kernel_size=[3, 3], strides = 2, padding="same", activation=None, name = "Conv8")
    if use_batch_norm:
        Conv8 = tf.contrib.layers.batch_norm(inputs = Conv8)
    Conv8 = tf.maximum(Conv8, alpha*Conv8)
    
    Conv9 = tf.layers.conv2d(inputs=Conv8, filters=16*filt, kernel_size=[3, 3], strides = 1, padding="same", activation=None, name = "Conv9")
    if use_batch_norm:
        Conv9 = tf.contrib.layers.batch_norm(inputs = Conv9)
    Conv9 = tf.maximum(Conv9, alpha*Conv9)
    Conv10 = tf.layers.conv2d(inputs=Conv9, filters=16*filt, kernel_size=[3, 3], strides = 2, padding="same", activation=None, name = "Conv10")
    if use_batch_norm:
        Conv10 = tf.contrib.layers.batch_norm(inputs = Conv10)
    Conv10 = tf.maximum(Conv10, alpha*Conv10)
    
    logits = tf.layers.conv2d(inputs=Conv10, filters=5, kernel_size=[3, 3], strides = 1, padding="same", activation=None, name = "Conv11")
    out = tf.nn.sigmoid(logits)
    return logits

x = tf.placeholder(tf.float32, shape=[None, 192, 256, 3])
logits = model(x, use_batch_norm=True)
saver = tf.train.Saver()
cap = cv2.VideoCapture(0)
with tf.Session() as sess:
    saver.restore(sess, "Saved models/face3")
    while True:
        ret, frame = cap.read()
        grid_logits = logits.eval(feed_dict={x: cv2.resize(frame, (256, 192)).reshape((1, 192, 256, 3))}).reshape((6, 8, 5))
        cv2.imshow('Sapphire 2: Inference mode', cv2.resize(DrawBoundingBoxes(cv2.resize(frame, (256, 192)), grid_logits),(640, 480)))
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break